# 支持向量机（SVM）和SMO算法
找到具有最小间隔的数据点，再对该间隔最大化：
$$
\mathop{\arg\max}_{(w,b)}\{\min_n \ label·\frac{(w^Tx+b)}{||w||}\}
$$
转化后的优化目标函数：
$$
\begin{align}
\max_\alpha[\sum_{i=1}^m\alpha-\frac{1}{2}\sum_{i,j=1}^{m}&label^{(i)}·label^{(j)}·\alpha_i·\alpha_j·<x^{(i)},x^{(j)}>] \\\\
st. \ &\alpha\geq0\\
&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$
引入松弛变量后约束条件变为：
$$
\begin{align}
st. \ &C\geq\alpha\geq0\\&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$

## SMO：序列最小优化（Sequential Minimal Optimization）
将大优化问题分解为多个小优化问题  
SMO目标：求出一系列$\alpha$和$b$，然后可以容易计算出$w$  
工作原理：  
1.每次循环选择两个$\alpha$进行优化处理  
2.一旦找到一对“合适”的$\alpha$，就一个增大一个减小 

In [2]:
import numpy as np
from numpy import *
from time import sleep
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# 读取数据，获得特征与标签
def loadDataSet(fileName):
    """
    读取数据集
    参数：
        fileName -- 文件名
    返回：
        dataMat -- 数据矩阵
        label -- 标签矩阵
    """
    dataMat = []; labelMat = []                                      #新建数据和标签矩阵列表
    fr = open(fileName)                                              #新建数据和标签矩阵列表
    for line in fr.readlines():                                      #一行行读取      
        lineArr = line.strip().split('\t')                           #以制表符分割        
        dataMat.append([np.float(lineArr[0]), np.float(lineArr[1])]) #数据第1、2列添加到数据矩阵       
        labelMat.append(np.float(lineArr[2]))                        #数据第3列添加到标签矩阵
    return dataMat,labelMat

In [4]:
def selectJrand(i,m):
    """
    随机选取和i不同的j
    参数:
        i -- 第一个alpha的下标
        m -- alpha的个数
    返回：
        j -- 随机的值
    """
    j=i
    while (j==i):
        j = np.int(np.random.uniform(0,m))
    return j

In [5]:
def clipAlpha(aj,H,L):
    """
    约束alpha
    参数：
        aj -- 迭代得到的未约束alpha的值
        H -- 上界
        L -- 下界
    返回：
        aj -- 约束后的alpha
    """
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

### 简化版SMO算法
- 伪代码:  
创建一个alpha向量并将其初始化为0向量  
当迭代次数小于最大迭代次数时（外循环）  
&ensp;&ensp;&ensp;&ensp;对数据集中的每个数据向量（内循环）：  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;如果该数据向量可以被优化：  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;随机选择另外一个数据向量  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;同时优化这两个向量  
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;如果两个向量都不能被优化，退出内循环  
&ensp;&ensp;&ensp;&ensp;如果所有向量都没被优化，增加迭代数目，继续下一次循环

In [9]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """
    简化的SMO算法
    参数：
        dataMatIn -- 输入数据矩阵
        classLabels -- 标签类列表
        C -- 权重超参数
        toler -- 容错率
        maxIter -- 最大迭代次数
    返回：
        b -- 更新的偏差参数
        alphas -- 更新的alpha参数向量
    """
    dataMatrix = np.array(dataMatIn); labelMat = np.array(classLabels).reshape(-1, 1)   #数据和标签转化为矩阵形式    
    b = 0; m,n = np.shape(dataMatrix)                                       #偏差参数初始化； #记录数据矩阵的维度                 
    alphas = np.zeros((m,1))                                                            #初始化alpha向量为全0
    iter = 0                                                                            #初始化迭代次数计数器   
    while (iter < maxIter):                                                             #当迭代次数小于最大迭代次数时
        alphaPairsChanged = 0                                                           #用于记录alpha修改的次数
        for i in range(m):                                                              #逐行遍历数据矩阵，寻找可优化的alpha
            fXi = np.float(np.dot((alphas*labelMat).T, np.dot(dataMatrix, dataMatrix[i:i+1,:].T))) + b      #i的预测类别
            Ei = fXi - np.float(labelMat[i, 0])                                                             #i的误差
                                                                                                            #检查是否有记录i违背KKT条件
            if ((labelMat[i, 0]*Ei < -toler) and (alphas[i, 0] < C)) or ((labelMat[i, 0]*Ei > toler) and (alphas[i, 0] > 0)):   #对于alphai<C,为落在最大分隔边界的支持向量，其yi*Fxi-1=0                                             
                j = selectJrand(i,m)                                                                                                #利用函数selectJrand随机选择记录j
                fXj = np.float(np.dot((alphas*labelMat).T, np.dot(dataMatrix, dataMatrix[j:j+1,:].T))) + b                          #j的预测类别
                Ej = fXj - np.float(labelMat[j, 0])                                                                                 #j的预测误差
                alphaIold = alphas[i, 0].copy(); alphaJold = alphas[j, 0].copy();                                                   #copy旧的alphai和alphaj
                if (labelMat[i, 0] != labelMat[j, 0]):                                                                              #如果i和j的真实类别标签不同
                    L = max(0, alphas[j, 0] - alphas[i, 0])                                                                         #下界L与上界H用于将alphaj调整在0到C之间
                    H = min(C, C + alphas[j, 0] - alphas[i, 0])
                else:                                                                                                               #如果i和j的真实类别标签相同
                    L = max(0, alphas[j, 0] + alphas[i, 0] - C)
                    H = min(C, alphas[j, 0] + alphas[i, 0])
                if L==H:  continue                                                                                                  #若上界等于下界，就执行下一次循序
                                                                                                                                    #计算最优修改量
                eta = 2.0 * np.dot(dataMatrix[i:i+1,:], dataMatrix[j:j+1,:].T) - np.dot(dataMatrix[i:i+1,:], dataMatrix[i:i+1,:].T) - np.dot(dataMatrix[j:j+1,:], dataMatrix[j:j+1,:].T)
                if eta >= 0:  continue
                alphas[j, 0] -= labelMat[j, 0]*(Ei - Ej)/eta                                                                        #更新alphaj，
                alphas[j, 0] = clipAlpha(alphas[j, 0],H,L)                                                                          #利用函数clipAlpha约束alphaj
                if (abs(alphas[j, 0] - alphaJold) < 0.00001):  continue                                                             #判断精度是否满足要求，是，则执行下一次循环
                alphas[i, 0] += labelMat[j, 0]*labelMat[i, 0]*(alphaJold - alphas[j, 0])                                            #修改i的大小和j相同，但是方向相反，保证约束条件成立
                                                                                                                                    #计算新的常数项b
                b1 = b - Ei- labelMat[i, 0]*(alphas[i, 0]-alphaIold)*np.dot(dataMatrix[i:i+1,:], dataMatrix[i:i+1,:].T) - labelMat[j, 0]*(alphas[j, 0]-alphaJold)*np.dot(dataMatrix[i:i+1,:], dataMatrix[j:j+1,:].T)
                b2 = b - Ej- labelMat[i, 0]*(alphas[i, 0]-alphaIold)*np.dot(dataMatrix[i:i+1,:], dataMatrix[j:j+1,:].T) - labelMat[j, 0]*(alphas[j, 0]-alphaJold)*np.dot(dataMatrix[j:j+1,:], dataMatrix[j:j+1,:].T)
                if (0 < alphas[i, 0]) and (C > alphas[i, 0]): b = b1
                elif (0 < alphas[j, 0]) and (C > alphas[j, 0]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1                                                                                             #更新alpha并增加更新次数记录
                #print("循环次数: {} alpha:{}, alpha对修改了 {} 次".format(iter,i,alphaPairsChanged))                                #打印更新次数
        if (alphaPairsChanged == 0): iter += 1                                                                                     #更新时增加迭代次数记录
        else: iter = 0
        #print("迭代次数: {}".format(iter))
    return b,alphas

In [12]:
#测试函数
dataArr, labelArr = loadDataSet("testSet.txt")
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)
print("b = {}".format(b))
print("大于0的alpha\n{}".format(alphas[alphas > 0]))

FileNotFoundError: [Errno 2] No such file or directory: 'testSet.txt'